In [1]:
def package_url_rent(page):
    return 'https://streeteasy.com/rentals='+ page

In [2]:
def package_url_sale(page):
    return 'https://streeteasy.com/for-sale/nyc=' + page

In [3]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import random
import json
import requests

price =[]
where =[]
bed = []
bath = []
size =[]
monthly = []
steet = []

In [4]:
for x in range(10): #loop through all pages
    url=package_url_rent(str(x))
    openurl= urllib.request.install_opener
    import urllib.request 
    soup = BeautifulSoup(html,'html.parser')
    lst = soup.find_all(lambda tag: tag.has_attr('data-id'))
    for i in range(len(lst)):
        #price
        if lst[i].find_all('span',{'class':'price'})==[]:
            price.append('')
        else:
            price.append(lst[i].find_all('span',{'class':'price'})[0].string)
        #bedroom
        if(lst[i].find_all('span',{'class':'first_detail_cell'})==[]):
            bed.append('')
        else:
            bed.append(lst[i].find_all('span',{'class':'first_detail_cell'})[0].string)
        #bedroom
        if(lst[i].find_all('span',{'class':'detail_cell'})==[]):
            bath.append('')
        else:
            bath.append(lst[i].find_all('span',{'class':'detail_cell'})[0].string)
        #size
        if(lst[i].find_all('span',{'class':'last_detail_cell'})==[]):
            size.append('')
        else:
            size.append(lst[i].find_all('span',{'class':'last_detail_cell'})[0].string)  
print 

SyntaxError: invalid syntax (<ipython-input-4-9f943bb26d56>, line 28)

In [ ]:
import pandas as pd
import numpy as np
data = {'price':price,'where':where,'bed':bed, 'bath':bath, 'size':size,'furnished':0}
df = pd.DataFrame(index=data, columns=['price', 'where', 'bed', 'bath','size','furnished'])

#move from size to bath
cond=[]
for i in data['size']:
    if(i==''):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
df.loc[cond,'bath']= df.loc[cond,'size'] 
df.loc[cond,'size']=''

#move from bed to bath
cond=[]
for i in data['bed']:
    if(i=='' or i=='Furnished' or i=='studio'):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
df.loc[cond,'bath']=df.loc[cond,'bed'] 
df.loc[cond,'bed']=''

#move from bath to bed
cond=[]
for i in data['bath']:
    if(i==''):
        cond.append(False)
    else:
        if(len(i.split(" "))==1):
            cond.append(True)
        else:
            if(i.split(" ")[1] in ('bath','baths')):
                cond.append(False)
            else:
                cond.append(True)
df.loc[cond,'bed']=df.loc[cond,'bath'] 
df.loc[cond,'bath']=''

#move from bed to size
cond=[]
for i in data['bed']:
    if(i=='' or i=='studio'):
        cond.append(False)
    else:
        if(i.split(" ")[1] in ('bed','beds')):
            cond.append(False)
        else:
            cond.append(True)
df.loc[cond,'size']=df.loc[cond,'bed'] 
df.loc[cond,'bed']=''


#replace blank with nan
data=df.applymap(lambda x: np.nan if x=='' else x)

#data
df.to_csv('rent.csv',encoding='utf-8')

In [ ]:
#size to numeric
cond=data['size'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'size']=int(data['size'][i].split(" ")[0].replace(',',''))
#bath to numeric
cond=data['bath'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'bath']=float(data['bath'][i].split(" ")[0].replace('+',''))
#bed to numeric
cond=data['bed'].isnull()
data['bed']=data['bed'].replace('studio','0 bed')
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'bed']=float(data['bed'][i].split(" ")[0].replace(',','').replace('+',''))

df.to_csv('rent_2.csv')